# Modules
In Python, a module must be defined in a dedicated file. In Julia, modules are independent from the file system. You can define several modules per file, or define one module across multiple files, it's up to you.

Or we can use `import` with a relative path with a dot `.` to indicate that we want the module located in the current module:

In short:

|Julia | Python
|------|-------
|`import Foo` | `import foo`
|`import Foo.Bar` | `from foo import bar`
|`import Foo.Bar: a, b` | `from foo.bar import a, b`
|`import Foo.Bar.a, Foo.Bar.b` | `from foo.bar import a, b`
|`import .Foo` | `import .foo`
|`import ..Foo.Bar` | `from ..foo import bar`
|`import ...Foo.Bar` | `from ...foo import bar`
|`import .Foo: a, b` | `from .foo import a, b`
||
|`using Foo` | `from foo import *; import foo`
|`using Foo.Bar` | `from foo.bar import *; from foo import bar `
|`using Foo.Bar: a, b` | `from foo.bar import a, b`

|Extending function `Foo.f()` | Result
|-----------------------------|--------
|`import Foo.f  # or Foo: f` <br />`f(x::Int64) = ...`  | OK
|`import Foo`<br />`Foo.f(x::Int64) = ...` | OK
|`using Foo`<br />`Foo.f(x::Int64) = ...` | OK
|`import Foo.f # or Foo: f`<br />`Foo.f(x::Int64) = ...` | `ERROR: Foo not defined`
|`using Foo`<br />`f(x::Int64) = ...` | `ERROR: Foo.f must be explicitly imported`
|`using Foo: f`<br />`f(x::Int64) = ...` | `ERROR: Foo.f must be explicitly imported`

# Scopes
Julia has two types of scopes: global and local.

Every module has its own global scope, independent from all other global scopes. There is no overarching global scope.

Modules, macros and types (including structs) can only be defined in a global scope.

Most code blocks, including `function`, `class`, etc., have their own local scope. For example:

In [2]:
for q in range(1, 4):
    print(q)

try:
    print(q) # q is available here
except Exception as ex:
    print(ex)

1
2
3
3


A local scope inherits from its parent scope:

In [3]:
z = 5
for i in range(1, 4):
    w = 10
    print(i * w * z) # i and w are local, z is from the parent scope

50
100
150


An inner scope can assign to a variable in the parent scope, if the parent scope is not global:

In [4]:
for i in range(1, 4):
    s = 0
    for j in range(1, 6):
        s = j # variable s is from the parent scope
    
    print(s)

5
5
5


You can't force a variable to be local in python.

To assign to a global variable, you must declare the variable as `global` in the local scope:

In [11]:
for i in range(1, 4):
    global p
    p = i

p

3

In functions, assigning to a variable which is not explicitly declared as global always makes it local:

In [13]:
s, t = 1, 2 # globals

def foo():
   s = 10 * t # s is local, t is global
   return s

print(foo())
print(s)

20
1


Just like in Python, functions can capture variables from the enclosing scope (not from the scope the function is called from):

In [18]:
t = 1

foo = lambda :t # foo() captures t from the global scope

def bar():
    t = 5 # this is a new local variable
    print(foo()) # foo() still uses t from the global scope

bar()

1


In [19]:
def quz():
    global t
    t = 5 # we change the global t
    print(foo()) # and this affects foo()

quz()

5


Closures work much like in Python:

In [23]:
def create_multiplier(n):
    def mul(x):
        return x * n # variable n is captured from the parent scope

    return mul

mul2 = create_multiplier(2)
print(mul2(5))

10


In this example, the first argument's default value is `a + 1`, where `a` comes from the parent scope (i.e., the global `a` in this case). However, the second argument's default value is `a`, where `a` in this case is the value of the first argument (<u>not</u> the parent scope's `a`).

Note that `if` blocks and `begin` blocks do <u>not</u> have their own local scope, they just use the parent scope:

In [34]:
a = 1
if True:
    a = 2 # same `a` as above
    
a

2